## Handle Pydantic Errors


#### 1. Required Field Errors


In [1]:
from pydantic import BaseModel, ValidationError


class Product(BaseModel):
    product_id: int
    name: str
    price: float
    is_active: bool = True  # Default value means this is not a missing field error

In [2]:
print("--- 1. Missing Required Field Error ---")
try:
    Product(name="Gadget X", price=99.99)
    # 'product_id' is missing
except ValidationError as e:
    # Print the full error for reference
    print("Full ValidationError:\n", e)
    print("\nFocused Error Details:")
    # Focus on the first error's location (loc) and message (msg)
    print(e.errors())
    for error_detail in e.errors():
        print(f"Location: {error_detail['loc']}")  # ('product_id',)
        print(f"Error Type: {error_detail['type']}")  # 'missing'
        print(f"Message: {error_detail['msg']}")

--- 1. Missing Required Field Error ---
Full ValidationError:
 1 validation error for Product
product_id
  Field required [type=missing, input_value={'name': 'Gadget X', 'price': 99.99}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

Focused Error Details:
[{'type': 'missing', 'loc': ('product_id',), 'msg': 'Field required', 'input': {'name': 'Gadget X', 'price': 99.99}, 'url': 'https://errors.pydantic.dev/2.12/v/missing'}]
Location: ('product_id',)
Error Type: missing
Message: Field required


In [3]:
print("\n--- 2. Type Mismatch Error ---")
try:
    Product(product_id="P101", name="Gadget Y", price="expensive")
    # 'product_id' should be int, 'price' should be float/number
except ValidationError as e:
    # Pydantic often coerces types, but "P101" cannot be coerced to an int
    print("Full ValidationError:\n", e)
    print("\nFocused Error Details (Multiple Errors):")
    # Show how to iterate through multiple errors
    for error in e.errors():
        print(f"  Field: {error['loc']}, Type: {error['type']}, Message: {error['msg']}")


--- 2. Type Mismatch Error ---
Full ValidationError:
 2 validation errors for Product
product_id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='P101', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing
price
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='expensive', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/float_parsing

Focused Error Details (Multiple Errors):
  Field: ('product_id',), Type: int_parsing, Message: Input should be a valid integer, unable to parse string as an integer
  Field: ('price',), Type: float_parsing, Message: Input should be a valid number, unable to parse string as a number


---

#### 2. Constraint and Custom Validator Errors


In [4]:
from pydantic import BaseModel, Field, ValidationError, field_validator


class Contact(BaseModel):
    # Constraint Error: email must match a pattern (regex)
    email: str = Field(..., pattern=r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}$")

    # Constraint Error: zip_code must be exactly 5 characters long
    zip_code: str = Field(..., min_length=5, max_length=5)

    # Custom Validator Error: ensure the name is capitalized
    name: str

    @field_validator("name")
    @classmethod
    def check_name_capitalized(cls, v: str) -> str:
        if v and not v[0].isupper():
            # Raise a simple ValueError, which Pydantic catches and converts
            raise ValueError("Name must start with a capital letter.")
        return v

In [5]:
print("--- 1. Constraint (Regex and Length) Error ---")
try:
    Contact(email="bad-email@", zip_code="12345678", name="Yash")
except ValidationError as e:
    print("Full ValidationError:\n", e)
    print("\nFocused Error Details:")
    # Both 'email' (regex) and 'zip_code' (length) fail
    for error in e.errors():
        print(f"  Field: {error['loc'][0]}, Message: {error['msg']}")

--- 1. Constraint (Regex and Length) Error ---
Full ValidationError:
 2 validation errors for Contact
email
  String should match pattern '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}$' [type=string_pattern_mismatch, input_value='bad-email@', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_pattern_mismatch
zip_code
  String should have at most 5 characters [type=string_too_long, input_value='12345678', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_long

Focused Error Details:
  Field: email, Message: String should match pattern '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}$'
  Field: zip_code, Message: String should have at most 5 characters


In [6]:
print("\n--- 2. Custom Validator Error ---")
try:
    Contact(email="test@example.com", zip_code="12345", name="yash")
    # 'bob' fails the custom capitalization check
except ValidationError as e:
    print("Full ValidationError:\n", e)
    print("\nFocused Error Details:")
    for error_detail in e.errors():
        # The message is the custom one raised by ValueError
        print(f"Field: {error_detail['loc'][0]}")
        print(f"Message: {error_detail['msg']}")
        print(f"Type: {error_detail['type']}")


--- 2. Custom Validator Error ---
Full ValidationError:
 1 validation error for Contact
name
  Value error, Name must start with a capital letter. [type=value_error, input_value='yash', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

Focused Error Details:
Field: name
Message: Value error, Name must start with a capital letter.
Type: value_error
